In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

## Chatbot

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, AIMessage

model = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    api_key=GEMINI_API_KEY,
)

In [2]:
model.invoke(
    [
        HumanMessage(content="my name is jignesh"),
        AIMessage(content="hello jignesh"),
        HumanMessage(content="What is my name?"),
    ]
)

AIMessage(content='Your name is Jignesh.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-1.5-flash', 'safety_ratings': []}, id='run--de4cea3e-3494-4aee-888f-7ee9d053cc4a-0', usage_metadata={'input_tokens': 15, 'output_tokens': 8, 'total_tokens': 23, 'input_token_details': {'cache_read': 0}})

In [3]:
from langgraph.graph import StateGraph, START, END
from langchain_core.messages import BaseMessage, AnyMessage
from langgraph.graph.message import add_messages
from typing import TypedDict, Annotated
from dataclasses import dataclass
from langgraph.checkpoint.memory import MemorySaver

checkpoint = MemorySaver()

@dataclass
class Messages:
    messages: Annotated[list[AnyMessage], add_messages]

async def call_model(messages: Messages):
    async for chunk in model.astream(messages.messages):
        yield {"messages": chunk}

workflow = (
    StateGraph(Messages)
    .add_node("call_model", call_model, output=BaseMessage, input=Messages)
    .add_edge(START, "call_model")
    .add_edge("call_model", END)
    .compile(checkpointer=checkpoint)
)

In [10]:
config = {"configurable": {"thread_id": "abc123"}}

async for i in  workflow.astream({"messages": ["my name is jignesh."]}, config, stream_mode="messages"):
    print(i[0].content)

It
's nice to meet you, Jignesh!  How can I help
 you today?

